In [6]:
import tensorflow as tf
import numpy as np
import zipfile
import librosa


# Download the model to yamnet-classification.tflite
interpreter = tf.lite.Interpreter(r"C:\Users\USER\Downloads\Speech Rec\lite-model_yamnet_classification_tflite_1.tflite")

input_details = interpreter.get_input_details()
waveform_input_index = input_details[0]['index']
output_details = interpreter.get_output_details()
scores_output_index = output_details[0]['index']

# # Input: 0.975 seconds of silence as mono 16 kHz waveform samples.
# waveform = np.zeros(int(round(0.975 * 16000)), dtype=np.float32)
# print(waveform.shape)  # Should print (15600,)

# Load audio file
audio_path = r'C:\Users\USER\Downloads\Compressed\AudioRecognition_py-main\DataTest\lion1.wav'
y, sr = librosa.load(audio_path, sr=16000, mono=True)
# Resample audio to 16kHz
y_resampled = librosa.resample(y, sr, 16000)
# Normalize audio waveform
y_norm = librosa.util.normalize(y_resampled)
print(y_norm.shape)



interpreter.resize_tensor_input(waveform_input_index, [y_norm.size], strict=False)
interpreter.allocate_tensors()
interpreter.set_tensor(waveform_input_index, y_norm)
interpreter.invoke()
scores = interpreter.get_tensor(scores_output_index)
print(scores.shape)  # Should print (1, 521)

top_class_index = scores.argmax()
labels_file = open(r'C:\Users\USER\Downloads\Speech Rec\labels.txt').readlines()
# labels = [l.decode('utf-8').strip() for l in labels_file.readlines()]
print(len(labels_file))  # Should print 521
print(labels_file[top_class_index])  # Should print 'Silence'.

ModuleNotFoundError: No module named 'librosa'

In [3]:
import cv2
import numpy as np
import pyaudio
import wave
import tensorflow as tf
import librosa

# Download the model to yamnet-classification.tflite
interpreter = tf.lite.Interpreter("/content/lite-model_yamnet_classification_tflite_1.tflite")

input_details = interpreter.get_input_details()
waveform_input_index = input_details[0]['index']
output_details = interpreter.get_output_details()
scores_output_index = output_details[0]['index']

# Set up audio stream
audio = pyaudio.PyAudio()
audio_frames = []

# Define audio stream parameters
format = pyaudio.paInt16
channels = 1
sample_rate = 16000
frame_size = 1024

stream = audio.open(format=format,
                    channels=channels,
                    rate=sample_rate,
                    frames_per_buffer=frame_size,
                    input=True,
                    stream_callback=callback)

# OpenCV video capture
cap = cv2.VideoCapture(0)

# Define video capture parameters
width = 640
height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Main loop
while True:
    ret, frame = cap.read()

    # Process the video frame here...

    # Capture audio frame
    audio_frames.append(frame.tobytes())

    # Audio recognition
    if len(audio_frames) > 0:
        audio_data = np.frombuffer(audio_frames[0], dtype=np.int16)

        # Resample audio to 16kHz
        y_resampled = librosa.resample(audio_data, orig_sr=sample_rate, target_sr=16000)
        # Normalize audio waveform
        y_norm = librosa.util.normalize(y_resampled)

        # Run inference
        interpreter.resize_tensor_input(waveform_input_index, [y_norm.size], strict=False)
        interpreter.allocate_tensors()
        interpreter.set_tensor(waveform_input_index, y_norm)
        interpreter.invoke()
        scores = interpreter.get_tensor(scores_output_index)

        top_class_index = scores.argmax()
        labels_file = open('/content/labels.txt').readlines()
        print(labels_file[top_class_index])

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
stream.stop_stream()
stream.close()
audio.terminate()


AttributeError: module 'tensorflow' has no attribute 'lite'